# RoBERTa

In [3]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 민서가 준 data.csv 파일

In [6]:
# data 변수명으로 csv 파일 열기
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/BITAMIN/24 nlp [projects]/주제별 카톡(1)/data.csv')
print(data)

      subject                                               text
0     타 국가 이슈  1 : 인도는 폭우가 와서 난리라네\n2 : 요새 그냥 지구가 난리다 ㅠㅠ\n3 :...
1          미용  1 : 다이어트가 최고의 성형이야!\n2 : 다이어트가 최고의 성형이지 하하\n3 ...
2          건강  1 : 건강 해야 해 친구들 우리 모두\n2 : 맞아 건강해야 해 하하\n3 : 그...
3      상거래 전반  1 : 요새 뭐가 젤 갖고 싶누 하하\n2 : 나는 루이비통 후드티 하하\n3 : ...
4       방송/연예  1 : 아 그거 기대작임 지리산\n2 : 지리산 이번 주에 함 하하\n2 : 완전 ...
...       ...                                                ...
4994   스포츠/레저  1 : 나 오늘부터 헬스 다닐거야!\n2 : 오 너 꾸준히 다닐 수 있음?\n1 :...
4995       게임  1 : 나 요즘 카트라이더가 왜 이리 재밌는지 모르겠어 하하\n2 : 너도 카트라이...
4996       여행  1 : 요즘 따라 너무 일본이 가고 싶어 ㅠㅠ\n2 : 너 일본 한 번도 안 가 봄...
4997    계절/날씨  1 : 나 어제 강릉 갔다 온 거 말했었나? 하하\n2 : 엉? 너 어제 강릉갔다 ...
4998     사회이슈  1 : 너 하루에 커피 몇 잔 마셔?\n2 : 나는 커피 안 좋아함.\n1 : 오 ...

[4999 rows x 2 columns]


In [7]:
type(data) # 데이터 프레임

pandas.core.frame.DataFrame

In [10]:
data.head()

,subject,text
0,타 국가 이슈,1 : 인도는 폭우가 와서 난리라네\n2 : 요새 그냥 지구가 난리다 ㅠㅠ\n3 :...
1,미용,1 : 다이어트가 최고의 성형이야!\n2 : 다이어트가 최고의 성형이지 하하\n3 ...
2,건강,1 : 건강 해야 해 친구들 우리 모두\n2 : 맞아 건강해야 해 하하\n3 : 그...
3,상거래 전반,1 : 요새 뭐가 젤 갖고 싶누 하하\n2 : 나는 루이비통 후드티 하하\n3 : ...
4,방송/연예,1 : 아 그거 기대작임 지리산\n2 : 지리산 이번 주에 함 하하\n2 : 완전 ...


## 전처리

In [25]:
# RoBERTa는 input으로 '문장'을 받음 -> 텍스트 데이터 토큰화

labels = data['subject'].values # 값만 따로 뽑아내기
texts = data['text'].values

In [26]:
labels

array(['타 국가 이슈', '미용', '건강', ..., '여행', '계절/날씨', '사회이슈'], dtype=object)

In [27]:
texts

array(['1 : 인도는 폭우가 와서 난리라네\n2 : 요새 그냥 지구가 난리다 ㅠㅠ\n3 : 인도는 요새 계속 난리네 ?\n1 : 사람이 벌써 30명이 죽었대!\n2 : 우리나라도 그렇고 하하\n3 : 헐 대박이네\n1 : 홍수가 나가꾸 ㅠ 난리 난리\n2 : 요새 왜이리 이상기후 심한 거 같니\n3 : 얼마나 났길 래 사람이죽어\n1 : 먼 사람이 그리 많이 죽었을꼬\n2 : 이러다가 진짜 멸망하는 거 아니냐고\n3 : 맞아 ㅜㅜ 너무 날씨 미쳤음\n1 : 거기 홍수 나면 답도 없을 듯\n2 : 욜로족으로 살아야 되나\n3 : 우리 살아 있을 때 멸망 안 하길\n1 : 엉 지구가 아파요다 완전\n2 : 플라스틱을 줄이라고 하하\n3 : 그래 ㅜㅜ\n3 : 우리도 그렇고 다들 그렇때\n1 : 근데 옛날에도 홍수는 낫지 않아 ?\n2 : 마지막으로 지구가 주는 경고야 이건\n3 : 맞지 ㅜㅜ플라스틱진짜 미쳤다\n3 : 요새 배달도그렇고\n1 : 지구 온난화가 영향이 있는 건가?\n2 : 구니까 우리하나하나가 노력해야함\n3 : 그렇지 않을까 키키',
       '1 : 다이어트가 최고의 성형이야!\n2 : 다이어트가 최고의 성형이지 하하\n3 : 웅 난 요새 다시 살찌는중 ㅜㅜ\n1 : **는 꾸준히 노력하더라구\n2 : ***도 요새 살 빠지고 이뻐짐\n3 : 응 근데 안 빠짐 키키\n3 : 비밀인데\n1 : 저녁 굶고 막 나름 노력해\n2 : **는 열심히 운동함 하하\n3 : 오 키키 진짜 키키 조만간 샐러드 파티 해야겠네\n1 : 키키 오 ** 오빠가 칭찬 해줬어 키키\n2 : ***랑 같이 하하\n3 : 키키 아 진짜\n3 : 같이 하면 좋지\n1 : ***가 와서 운동은 열심히 해...\n2 : 꽤나 많이 빠짐 하하\n3 : 우리는 운동을 같이 안 해\n1 : 근데 처갓집 넘 빠져서...\n2 : ***는 근데 안 빠짐 하하\n3 : 아 처갓집 키키 더 열심히 운동하면 대\n1 : 처갓집이 지금 다이어트의 적이야 키키\n2 : 처갓집 꼭 먹어보

In [34]:
# 일부 데이터만 모델 돌리기
labels = data['subject'].head(100).values
texts = data['text'].head(100).values

In [35]:
# 데이터셋 분할(train, validation)

from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

## RoBERTa 모델링

In [36]:
# transformers 라이브러리 사용 -> transformers와 torch 라이브러리 설치
!pip install transformers torch # 완료

In [39]:
# Hugging Face 의 transformeers 라이브러리를 사용해 RoBERTa 전용 토크나이저 만들기
from transformers import RobertaTokenizer

# RoBERTa-base 모델의 토크나이저 로드
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# 토큰화
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)


^ 설명

<`from_pretrained` 메서드>
- Hugging Face의 `transformers` 라이브러리에서 제공하는 기능으로, 미리 학습된 사전 학습 모델을 불러오는 데 사용된다.
- `모델이름('roberta-base')`
    - 이 메서드에서 `roberta-base` 모델을 사용해서 미리 학습된 RoBERTa-base 모델을 불러온다. 이 모델을 일반적으로 영어 텍스트로 학습된 사전 학습 모델이다. *(흠.. 그렇다면 한국어로 학습되어있는걸 사용해야하지 않을까?)*


In [40]:
# 데이터셋 준비

import torch

# `TextDataset` 클래스 정의
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# `train_dataset` 및 `val_dataset` 인스턴스 생성
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)


^

PyTorch의 `Dataset` 클래스를 사용해 학습과 검증에 사용할 데이터셋 준비.
  
1. `TextDataset` 클래스 정의
- PyTorch의 `Dataset` 클래스를 상속하여 데이터셋을 구성
- 이렇게 하면, 모델 학습 시 데이터가 필요할 때마다 하나씩 샘플을 제공하여 효율적으로 데이터를 로드할 수 있음
2. `train_dataset` 및 `val_dataset` 인스턴스 생성

In [41]:
# 모델 로드 및 설정
# RoBERTa 모델을 불려오고, 텍스트 분류 작업을 위해 분류 헤드를 추가

from transformers import RobertaForSequenceClassification

# 2개 이상의 주제나 감정 범주에 따라 num_labels 값을 조정
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
# 학습 설정 및 훈련
# 학습을 위한 `Trainer`와 `TrainingArguments` 설정

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # 출력 폴더
    num_train_epochs=3,              # 학습 반복 횟수
    per_device_train_batch_size=8,   # 학습 배치 크기
    per_device_eval_batch_size=8,    # 평가 배치 크기
    warmup_steps=500,                # 워밍업 단계
    weight_decay=0.01,               # 가중치 감쇠
    logging_dir='./logs',            # 로깅 폴더
    report_to="none"                 # W&B 비활성화
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()


TypeError: new(): invalid data type 'str'

In [ ]:
# 모델 평가
# 학습이 완료되면 검증 데이터셋을 사용하여 모델을 평가한다.

eval_results = trainer.evaluate()
print(eval_results)


In [ ]:
# 새로운 텍스트에 대한 예측

test_text = ["예시 문장입니다."]
test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
outputs = model(**test_encodings)
predictions = torch.argmax(outputs.logits, dim=-1)
print("예측 결과:", predictions)


# KoRoBERTa
- RoBERTa가 영어 학습에 최적화되어있기 때문에 성능이 영어 데이터를 사용할 때보다 더 낮게 나타날 수 밖에 없다고 생각했다.
- 한국어 자연어 처리(NLP)를 위해 한국어 데이터로 학습된 RoBERTa 모델이다.
    - 한국어 문법과 어휘의 특성을 반영하도록 대규모 한국어 데이터로 사전 학습되었다.

- `koichi-san/korean-roberta-base`
    - 다른 모델 `KLUE-RoBERTa`나 `KoBERT` 등과 비교했을 때 이 모델은 중간 크기의 모델로, 학습 속도와 메모리 사용 측면에서 효율적이다.
    - 다양한 한국어 데이터(일상적인 대화, 뉴스기사 등)를 사용하여 사전 학습되었기 때문에 도메인 의존성이 크지 않고 여러 분야의 텍스트에 적합하게 사용할 수 있다.
    

In [50]:
# 필요한 라이브러리 불러오기
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

# 1. 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/BITAMIN/24 nlp [projects]/주제별 카톡(1)/data.csv')

# 변수명 'text'와 'label'로 설정
texts = data['text'].values
labels = data['subject'].values

# 2. 레이블을 숫자로 변환 (예: "건강" -> 0, "여행" -> 1, ...)
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
labels = [label_mapping[label] for label in labels]

# 3. 데이터셋 분할
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# 4. KoRoBERTa 토크나이저와 모델 불러오기
model_name = "korca/koroberta-base-lkm" # 원래는 koichi-san/korean-roberta-base
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_labels = len(label_mapping)  # 주제의 개수에 따라 num_labels 설정
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 5. 토큰화
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# 6. 데이터셋 준비
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

# 7. 학습 설정
training_args = TrainingArguments(
    output_dir='./results',          # 출력 폴더
    num_train_epochs=3,              # 학습 반복 횟수
    per_device_train_batch_size=8,   # 학습 배치 크기
    per_device_eval_batch_size=8,    # 평가 배치 크기
    warmup_steps=500,                # 워밍업 단계
    weight_decay=0.01,               # 가중치 감쇠
    logging_dir='./logs',            # 로깅 폴더
    evaluation_strategy="epoch",     # 각 에포크마다 평가
    report_to="none"                 # W&B 비활성화
)

# 8. Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# 9. 모델 학습
trainer.train()

# 10. 평가
eval_results = trainer.evaluate()
print("평가 결과:", eval_results)

# 11. 새로운 텍스트 예측
test_text = ["이것은 예시 문장입니다."]
test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
outputs = model(**test_encodings)
predictions = torch.argmax(outputs.logits, dim=-1)
print("예측 결과:", predictions)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at korca/koroberta-base-lkm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.547700,0.622893
2,0.453600,0.603380
3,0.186400,0.513530


평가 결과: {'eval_loss': 0.5135300159454346, 'eval_runtime': 7.1673, 'eval_samples_per_second': 139.522, 'eval_steps_per_second': 17.44, 'epoch': 3.0}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [51]:
# 입력 데이터와 모델이 서로 다른 장치(CPU와 GPU)에 있어서 발생한 오류
# 이 문제를 해결하려면 모델과 입력 데이터 모두를 동일한 장치(GPU 또는 CPU)로 이동시켜야 한다는데...

# CPU로 옮긴...
import torch

# 현재 장치를 CPU로 설정
device = torch.device("cpu")  # GPU 대신 CPU로 강제 설정

# 모델을 CPU로 이동
model.to(device)  # 모델을 CPU에 위치시킵니다.

# 예측 부분
test_text = ["이것은 예시 문장입니다."]
test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings = {key: val.to(device) for key, val in test_encodings.items()}  # 입력 데이터를 CPU로 이동

# 예측 실행
outputs = model(**test_encodings)
predictions = torch.argmax(outputs.logits, dim=-1)
print("예측 결과:", predictions)



예측 결과: tensor([4])


In [52]:
# 예측 결과 해석하기

# 기존의 label_mapping을 뒤집어 reverse_label_mapping을 생성
reverse_label_mapping = {idx: label for label, idx in label_mapping.items()}

# 예측 결과 해석
prediction_label = predictions.item()  # tensor([4])에서 숫자 4를 추출
predicted_topic = reverse_label_mapping[prediction_label]  # 숫자 4에 해당하는 주제 이름 찾기

print("예측된 주제:", predicted_topic)



예측된 주제: 사회이슈


### KoBERTa를 주제 생성에 그대로 사용할 수 없는 이유
KoBERTa는 **인코더 기반의 모델**로, 텍스트를 입력 받아 문맥을 이해하고 의미를 파악하는 데 적합하다. 하지만 KoBERTa는 생성 모델이 아니므로, 주제 생성 작업에는 한계가 있다.

- 잠깐 생각한건
    - KoBERTa로 텍스트 요약을 한다. 이걸 training data로 사용하고
    - 이걸 t5에 학습시킨다?

- 대화 주제 생성을 할 수 있는 KoBART를 사용해보자!

# KoBART
: 한국어 텍스트 생성에 최적화된 BART 모델   
(BART = BERT + GPT)
- 인코더-디코더 구조  
:인코더-디코더 구조를 사용하여 텍스트를 입력받아 문맥을 이해한 후, 디코더가 이를 기반으로 새로운 텍스트를 생성한다. 이는 텍스트 요약이나 주제 생성처럼, 기존 정보를 요약하거나 압축하여 간단한 형태로 만들어야 하는 작업에서 강력한 성능을 보인다.

In [58]:
# 1. KoBART 모델과 토크나이저 불러오기
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

model_name = "gogamza/kobart-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 2. 주제 생성 함수 정의
def generate_topic(text):
    # 입력 텍스트 토큰화
    inputs = tokenizer([text], max_length=1024, return_tensors="pt", truncation=True)

    # KoBART 모델을 통해 주제 생성
    output_ids = model.generate(
        inputs['input_ids'],
        num_beams=5,                 # 빔 수를 높여 다양성 확보
        max_length=50,               # 출력될 최대 토큰 길이
        early_stopping=True,         # 의미 있는 지점에서 멈춤
        no_repeat_ngram_size=2,      # 반복 방지를 위해 n-gram 반복 금지
        do_sample=True,              # 샘플링 모드 활성화
        temperature=0.7,             # 온도를 설정하여 결과 다양성 제어
        top_k=50,                    # 상위 k개의 단어에서 선택
        top_p=0.92                   # 누적 확률 0.92에 해당하는 단어들 중에서 선택
    )

    # 생성된 텍스트 디코딩
    topic = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return topic

# 3. 주제 생성 테스트
test_text = data['text'][0]
predicted_topic = generate_topic(test_text)
print("생성된 주제:", predicted_topic)





You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


생성된 주제: 1 : 그냥 지구가 난리다  
3 : 인도는 요새 계속 난리네?
1 : 사람이 벌써 30명이 죽었대!
2 : 우리나라도 그렇고 하하
 3 : 헐 대박이네



In [59]:
data['text'][0]

'1 : 인도는 폭우가 와서 난리라네\n2 : 요새 그냥 지구가 난리다 ㅠㅠ\n3 : 인도는 요새 계속 난리네 ?\n1 : 사람이 벌써 30명이 죽었대!\n2 : 우리나라도 그렇고 하하\n3 : 헐 대박이네\n1 : 홍수가 나가꾸 ㅠ 난리 난리\n2 : 요새 왜이리 이상기후 심한 거 같니\n3 : 얼마나 났길 래 사람이죽어\n1 : 먼 사람이 그리 많이 죽었을꼬\n2 : 이러다가 진짜 멸망하는 거 아니냐고\n3 : 맞아 ㅜㅜ 너무 날씨 미쳤음\n1 : 거기 홍수 나면 답도 없을 듯\n2 : 욜로족으로 살아야 되나\n3 : 우리 살아 있을 때 멸망 안 하길\n1 : 엉 지구가 아파요다 완전\n2 : 플라스틱을 줄이라고 하하\n3 : 그래 ㅜㅜ\n3 : 우리도 그렇고 다들 그렇때\n1 : 근데 옛날에도 홍수는 낫지 않아 ?\n2 : 마지막으로 지구가 주는 경고야 이건\n3 : 맞지 ㅜㅜ플라스틱진짜 미쳤다\n3 : 요새 배달도그렇고\n1 : 지구 온난화가 영향이 있는 건가?\n2 : 구니까 우리하나하나가 노력해야함\n3 : 그렇지 않을까 키키'